In [1]:
#https://www.youtube.com/watch?v=Y7XBsFzByTQ

import sys
import cv2
import os
import numpy
import pytesseract
import matplotlib.pyplot as plt

In [22]:
datasets = '/home/mostafa/Desktop/OCR/face/datasets'
 
# Part 1: Create fisherRecognizer
print('Recognizing Face Please Be in sufficient Lights...')
 
# Create a list of images and a list of corresponding names
(images, labels, names, id) = ([], [], {}, 0)
for (subdirs, dirs, files) in os.walk(datasets):
    for subdir in dirs:
        print('analysing dirs',dirs,'opening',subdir)
        names[id] = subdir #the name comes from the subdir
        subjectpath = os.path.join(datasets, subdir)
        for filename in os.listdir(subjectpath):
            path = subjectpath + '/' + filename
            images.append(cv2.imread(path, 0)) #the image comes from the path
            labels.append(int(id)) #label is the id
            print('reading from path:',path,' in subdir ',subdir,' with label: ',id)
        id += 1
(width, height) = (130, 100)

Recognizing Face Please Be in sufficient Lights...
analysing dirs ['man2', 'a man', 'man3', 'musk', 'mostafa'] opening man2
reading from path: /home/mostafa/Desktop/OCR/face/datasets/man2/3.jpg  in subdir  man2  with label:  0
reading from path: /home/mostafa/Desktop/OCR/face/datasets/man2/1.jpg  in subdir  man2  with label:  0
analysing dirs ['man2', 'a man', 'man3', 'musk', 'mostafa'] opening a man
reading from path: /home/mostafa/Desktop/OCR/face/datasets/a man/2.jpeg  in subdir  a man  with label:  1
reading from path: /home/mostafa/Desktop/OCR/face/datasets/a man/1.jpeg  in subdir  a man  with label:  1
analysing dirs ['man2', 'a man', 'man3', 'musk', 'mostafa'] opening man3
reading from path: /home/mostafa/Desktop/OCR/face/datasets/man3/1.jpg  in subdir  man3  with label:  2
reading from path: /home/mostafa/Desktop/OCR/face/datasets/man3/2.jpg  in subdir  man3  with label:  2
analysing dirs ['man2', 'a man', 'man3', 'musk', 'mostafa'] opening musk
reading from path: /home/mostafa

In [24]:
# Create a Numpy array from the two lists above
(images, labels) = [numpy.array(lis) for lis in [images, labels]]

In [25]:
# OpenCV trains a model from the images
# NOTE FOR OpenCV2: remove '.face'
model = cv2.face.LBPHFaceRecognizer_create() 

model.train(images, labels)

In [26]:
# Load the cascade
face_cascade = cv2.CascadeClassifier('/home/mostafa/Desktop/OCR/face/haarcascade_frontalface_default.xml')

In [27]:
cap=cv2.VideoCapture(-1)
if not cap.isOpened():
    print('VideoCapture -1 failed')
    cap=cv2.VideoCapture(0)

if not cap.isOpened():
    print('VideoCapture 0 failed')
    cap=cv2.VideoCapture(1)
else:
    print('VideoCapture 0 successfully done')
    
if not cap.isOpened():
    raise IOError("Cannot Open Video")
    
while True:
    # Read the input image
    ret,frame = cap.read()
    # Convert into grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Detect faces
    faces = face_cascade.detectMultiScale(gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )
    Report='found '+str(len(faces))+' faces'
    # Draw rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        # Try to recognize the face
        face = gray[y:y + h, x:x + w]
        face_resize = cv2.resize(face, (width, height))
        (id,prediction) = model.predict(face_resize)
        
        if prediction<500:
            cv2.putText(frame, '% s - %.0f' %(names[id], prediction), (x-10, y-10),cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0))
            Report=Report+", detecting "+names[id]
        else:
            cv2.putText(frame, 'not recognized',(x-10, y-10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0))

    cv2.putText(frame, 'Report: '+Report,(10, 18), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0))
    cv2.imshow('img', frame)
    
    # Display the output
    #cv2.imshow('img', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

VideoCapture 0 successfully done


In [ ]:
while False:
    ret,frame = cap.read()
    cntr=cntr+1
    if (cntr%10)==0:
        imgH,imgW,_=frame.shape
        x1,y1,w1,h1=0,0,imgH,imgW
        imgChar=pytesseract.image_to_string(frame)
        imgBoxes=pytesseract.image_to_boxes(frame)
        
        for boxes in imgBoxes.splitlines():
            boxes=boxes.split(' ')
            x,y,w,h=int(boxes[1]),int(boxes[2]),int(boxes[3]),int(boxes[4])
            cv2.rectangle(frame,(x,imgH-y),(w,imgH-h),(0,0,255),1)
            
        cv2.putText(frame,imgChar, (x1+int(w1/50),y1+int(h1/50)),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,0,0),2)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.imshow('Text detaction',frame)
    if cv2.waitKey():
        break
cap.release()
cv2.destroyAllWindows()